In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import regex

In [110]:
authors = pd.read_csv('authors_all.csv', encoding = 'utf-8', sep = ";")

In [218]:
authors.head()

,Id artist,name,name extended,type,Birth year,Death year,Birth city,Birth state,Birth country,Death city,Death state,Death country,Gender,Nationality (original),Nationality_Clean,ID artworks,average_year,artworks_creation_years,Nationality_Test
0,380000000034856,& IL TOPO,Aujourd’hui & IL TOPO réunit : Armando della V...,groupe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,groupe,NaN,None,380000000034695|380000000035465,2015.0,2015|2015,NaN
1,6664664,(À L'EMBALLAGE DE CLÉMENTINE),NaN,anonyme,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,280000000115916,NaN,NaN,NaN
2,6663648,(D'APRÈS),NaN,anonyme,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,540000000000477|540000000000478|54000000000047...,1959.0,1959|1959|1959|1959|1959|1959|1959|1959|1959|1...,NaN
3,6660426,(INSCRIPTIONS INTRAN STUDIO),NaN,anonyme,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,480000000000233,NaN,NaN,NaN
4,6660563,(PABLO PICASSO (D'APRÈS)),NaN,anonyme,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,160000000001109|160000000105183|160000000105186,1959.0,1948|1962|1967,NaN


In [112]:
authors['Nationality (original)'].value_counts().head()

française      17026
américaine      1536
italienne        757
britannique      611
allemande        573
Name: Nationality (original), dtype: int64

In [113]:
#Make new column for current nationalities
authors.insert(loc = 14, column ='Nationality_Clean', value = np.nan)

In [114]:
#Make new column for testing values
authors['Nationality_Test'] = authors['Nationality (original)']

## Find cases with only one nationality

In [115]:

def one_word(string):
    if string is not np.nan:
        string = str(string)
        if len(string.split()) <= 1:
            return str(string)
        else:
            return np.nan
    

In [116]:
authors['Nationality_Clean'] = authors['Nationality (original)'].apply(one_word)

## Cases with 1 nationality and () eg. 'allemande (RFA)'

In [122]:
pattern_1 = re.compile(r"([a-zA-ZÀ-ÿ'\-]+) (?:\()(?:RFA)? ?(?:Gibraltar)? ?(?:(?:[Rr]épublique) ?(?:démocratique)?(?:islamique ?(?:d'Iran)?)?(?:du)? ?(?:Vietnam)?)?(?:RDA)?(?:Taïwan)?(?:entre)? ?(?:après)?(?:avant)? ?(?:\d+)? ?(?:et)? ?(?:\d+)?\)$", flags = re.U)

In [123]:
filter_1 = authors['Nationality (original)'].str.contains(pattern_1, na = False)

C:\Users\antoine.trouche\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


In [124]:
#Get nationalities
result = authors['Nationality (original)'].str.extract(pattern_1, expand = False)

In [125]:
result = result.dropna()

In [126]:
#Insert strings with nationalities into the dataframe
authors.loc[filter_1, 'Nationality_Clean'] = result

In [127]:
#Insert strings with nationalities into the TEST dataframe
authors.loc[filter_1, 'Nationality_Test'] = result

In [128]:
authors[filter_1].shape

(907, 19)

In [ ]:
Examples of cases captured:

allemande (RFA)
russe (après 1992)
russe (avant 1918)
roumaine (avant 1948)
roumaine (entre 1948 et 1965)
chinoise (Taïwan)
allemande (RDA)
iranienne (république islamique d'Iran)
britannique (Gibraltar)
vietnamienne (République du Vietnam)
vietnamienne (république démocratique)
dominicaine (République)

## Find cases with 'et' eg. 'croate et française'

#No distinction between double nationality and sequence

In [130]:
pattern_2 = re.compile(r"([a-zA-ZÀ-ÿ'\-]+) (?:\((?:[\w'\-]+)? ?(?:\d+)?\))? ?(?:et) ([a-zA-ZÀ-ÿ''\-]+)", flags = re.U)

In [131]:
filter_2 = authors['Nationality (original)'].str.contains(pattern_2, na = False)

C:\Users\antoine.trouche\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


In [132]:
result = authors['Nationality (original)'].str.extract(pattern_2, expand=False)

In [133]:
result = result.dropna()

In [134]:
#Join nationalities into string 
result = result[result.columns[0]] + '|' + result[result.columns[1]]

In [135]:
#Insert strings with nationalities into the dataframe
authors.loc[filter_2, 'Nationality_Clean'] = result

In [136]:
#Insert strings with nationalities into the TEST dataframe
authors.loc[filter_2, 'Nationality_Test'] = result

In [137]:
authors.loc[filter_2].shape

(262, 19)

In [ ]:
Examples of cases captured:

russe (apres 1992) et americaine
yougoslave (avant 1991) et slovene
chilienne (avant 1991) et slovene
allemande (RFA) et autrichienne 
chilienne et francaise
russe (apres 1992) et americaine
francaise et finlandais
américaine et française (américaine à la naissance)
russe (avant 1918) et française


## Cases with two nationalities + 'à la naissance'

In [209]:
pattern_3 = re.compile(r"([\w'\-]+) ?(?:\([\w'\-]+\))?(?:de)? ?(?:\d+)? ?(?:à)? (?:\(avant \d+\))? ?(?:\d+)?(?:depuis \d+)?(?:\(avant \d+\))? ?(?:\()?([\w'\-]+) ?(?:\((?:[\w'\-]+) (?:[\w'\-]+)? (?:[\w'\-]+)?\))? ?\(?(?:avant)?(?:après)?(?:entre)? ?(?:\d+)? ?(?:et)? ?(?:\d+)?\)? ?(?:à la naissance)+\)?", flags = re.U)

In [210]:
filter_3 = authors['Nationality (original)'].str.contains(pattern_3, na = False)

C:\Users\antoine.trouche\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


In [211]:
#Get nationalities
result = authors['Nationality (original)'].str.extract(pattern_3, expand = False)

In [212]:
result = result.dropna()

In [213]:
#Join nationalities into string in correct sequence
result = result[result.columns[1]] + '|' + result[result.columns[0]]

In [214]:
#Insert strings with nationalities into the dataframe
authors.loc[filter_3, 'Nationality_Clean'] = result

In [215]:
#Insert strings with nationalities into the TEST dataframe
authors.loc[filter_3, 'Nationality_Test'] = result

In [217]:
authors[filter_3].shape

(834, 19)

In [ ]:
Examples of cases captured:
    
americaine depuis 1954 (yougoslave (avant 1991) à la naissance)
allemande (avant 1949) (suisse à la naissance)
israelienne (allemande à la naissance)
francaise de 1909 à 1953 (belge à la naissance)
francaise (roumaine (entre 1948 et 1965) à la naissance)
francaise (russe (après 1992) à la naissance)
francaise depuis 1970 (hongroise à la naissance)
francaise depuis 1927 (russe (avant 1918) à la naissance)
allemande (RFA) (bulgare à la naissance)
française (iranienne (république islamique d'Iran) à la naissance)
allemande (avant 1949) depuis 1914 (italienne à la naissance)


## Cases with two 'à la naissance' 

In [146]:
pattern_4 = re.compile(r"([a-zA-ZÀ-ÿ'\-]+) (?:\(?(?:[\w'\ -]+)?\)?)? ?\(?(?:à la naissance)\)? ?\(([a-zA-ZÀ-ÿ'\-]+) (?:à la naissance)\)?", flags = re.U)

In [147]:
filter_4 = authors['Nationality (original)'].str.contains(pattern_4, na = False)

C:\Users\antoine.trouche\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


In [148]:
#Get nationalities
result = authors['Nationality (original)'].str.extract(pattern_4, expand = False)

In [149]:
result = result.dropna()

In [150]:
#Join nationalities into string 
result = result[result.columns[0]] + '|' + result[result.columns[1]]

In [151]:
#Insert strings with nationalities into the dataframe
authors.loc[filter_4, 'Nationality_Clean'] = result

In [152]:
#Insert strings with nationalities into the TEST dataframe
authors.loc[filter_4, 'Nationality_Test'] = result

In [153]:
authors[filter_4].shape

(7, 19)

In [ ]:
Examples of cases captured:
    
argentine à la naissance (irlandaise à la naissance)
austro-hongroise à la naissance (espagnole à la naissance)
russe (avant 1918) à la naissance (norvégienne à la naissance)

## Cases with three nationalities

In [154]:
pattern_5 = re.compile(r"^(\b[a-zA-ZÀ-ÿ'\-]+\b)\s?(?:depuis \d+)?\s\((\b[a-zA-ZÀ-ÿ'\-]+\b)\s?\(?(?:avant|,|depuis|après|et)(?:\s\d+)?\)?\s?(?:depuis|après|et)?\s?(?:\d+)?\)?,?\s?(\b[a-zA-ZÀ-ÿ'\-]+\b)\s?\(?(?:avant|après)?\s?(?:\d+)?\)?\s(?:à la naissance)\)$", flags = re.U)

In [155]:
filter_5 = authors['Nationality (original)'].str.contains(pattern_5, na = False)

C:\Users\antoine.trouche\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


In [156]:
#Get nationalities
result = authors['Nationality (original)'].str.extract(pattern_5, expand = False)

In [157]:
result = result.dropna()

In [158]:
#Join nationalities into string in correct order
result = result[result.columns[2]] + '|' + result[result.columns[1]] + '|' + result[result.columns[0]]

In [159]:
#Insert strings with nationalities into the dataframe
authors.loc[filter_5, 'Nationality_Clean'] = result

In [160]:
#Insert strings with nationalities into the TEST dataframe
authors.loc[filter_5, 'Nationality_Test'] = result

In [161]:
authors[filter_5].shape

(16, 19)

In [ ]:
Examples of cases captured:
    
australienne depuis 1939 (chinoise et soviétique à la naissance)
francaise (israelienne, russe (après 1992) à la naissance)
mauricienne (francaise, britannique à la naissance)
francaise depuis 1960 (argentine depuis 1950, hongroise (avant 1946) à la naissance)
francaise depuis 1939 (allemande (avant 1949) depuis 1928, russe (avant 1918) à la naissance)
américaine depuis 1978 (mexicaine depuis 1941, espagnole à la naissance)
américaine (russe (avant 1918) et canadienne à la naissance)
française depuis 1960 (argentine depuis 1950, hongroise (avant 1946) à la naissance)
américaine (française, tchèque à la naissance) 	
    


## Check Test_Nationalities for not captured cases or mistakes

In [164]:
nv = authors.Nationality_Test.value_counts().rename_axis('Nationality').reset_index(name='frequency')

In [165]:
nv.to_csv('nationalities_new.csv', sep=',', encoding = 'utf-8-sig')

In [ ]:
Unique cases:
    
britannique de Hong Kong
(allemande à la naissance)
RFA|allemande
avant|russe
hongroise (entre 1946 et 1989) et néerlandaise
française de 1945 à 1949 (roumaine (avant 1948) jusqu'en 1945)
Kong|américaine
française de 1927 à 1970 (grecque de 1889 à 1927)
roumaine (entre 1948 et 1965) (allemande)
(sud-coréenne à la naissance)
                          
                          


## Handle unique cases and mistakes

In [166]:
authors.loc[authors.Nationality_Test == '(allemande à la naissance)', 'Nationality_Clean'] = 'allemande'

In [167]:
authors.loc[authors.Nationality_Test == 'britannique de Hong Kong', 'Nationality_Clean'] = 'britannique de Hong Kong'

In [168]:
authors.loc[authors.Nationality_Test == 'RFA|allemande', 'Nationality_Clean'] = 'allemande'

In [169]:
authors.loc[authors['Nationality (original)'] == 'américaine (allemande (RFA) à la naissance)', 'Nationality_Clean'] = 'allemande|américaine'

In [170]:
authors.loc[authors.Nationality_Test == 'avant|russe', 'Nationality_Clean'] = 'russe|allemande|française'

In [171]:
authors.loc[authors.Nationality_Test == 'hongroise (entre 1946 et 1989) et néerlandaise', 'Nationality_Clean'] = 'hongroise|néerlandaise'

In [172]:
authors.loc[authors.Nationality_Test == 'française de 1945 à 1949 (roumaine (avant 1948) jusqu\'en 1945)', 'Nationality_Clean'] = 'roumaine|française'

In [173]:
authors.loc[authors.Nationality_Test == 'Kong|américaine', 'Nationality_Clean'] = 'britannique de Hong Kong|américaine'

In [176]:
authors.loc[authors.Nationality_Test == 'française de 1927 à 1970 (grecque de 1889 à 1927)', 'Nationality_Clean'] = 'grecque|française'

In [179]:
authors.loc[authors.Nationality_Test == 'roumaine (entre 1948 et 1965) (allemande)', 'Nationality_Clean'] = 'roumaine|allemande'

In [182]:
authors.loc[authors.Nationality_Test == '(sud-coréenne à la naissance)', 'Nationality_Clean'] = 'sud-coréenne'

## Count 'française'

In [208]:
# Total number of artist having French nationality (including double)
authors[authors['Nationality_Clean'].str.contains(r'[|]?\bfrançaise\b[|]?', na = False)].shape[0]

17779

In [197]:
# Artists having only French nationality
authors['Nationality_Clean'].value_counts().head()

française      17026
américaine      1536
allemande       1051
italienne        757
britannique      612
Name: Nationality_Clean, dtype: int64

In [184]:
authors['Nationality (original)'].isnull().sum()

8494

In [185]:
authors.Nationality_Clean.isnull().sum()

8494

In [205]:
nationalities_clean = authors.Nationality_Clean.value_counts().rename_axis('Nationality').reset_index(name='frequency')

In [206]:
nationalities_clean.to_csv('nationalities_clean.csv', sep=',', encoding = 'utf-8-sig', index=False)